In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from datetime import datetime
import random
import re
import time
import undetected_chromedriver as uc
from selenium import webdriver
import json

In [10]:
def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36')
  options.add_argument("--incognito")
  driver = uc.Chrome(options = options)
  driver.set_page_load_timeout(60)
  driver.set_window_size(1366, 768)
  driver.set_window_position(0, 0)
  driver.delete_all_cookies()
  driver.execute_script("localStorage.clear();")
  driver.execute_script("sessionStorage.clear();")
  driver.execute_script("history.go(0);")
  driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
  })
  return driver

def get_cars(driver, urls):
    cars = []
    for url in urls:
        if url != None:
            car = get_car(driver, url)
            cars.append(car)
            cars_total += 1
    display(len(cars))
    display(cars)
    return cars

def get_links(driver, url):
    actions = ActionChains(driver)
    driver.get(url)
    time.sleep(random.uniform(0.5, 1.5))
    try:
        button = driver.find_element(by=By.XPATH, value='/html/body/div/div/div[2]/div/div[2]/div/div/div/footer/div/button[2]')
        actions.move_to_element(button).perform()
        button.click()
        time.sleep(random.uniform(0.8, 1.2))
    except:
        pass
    increment = 100
    position = 0
    while True:
        driver.execute_script("window.scrollTo(0, " + str(increment) + ");")
        elements = driver.find_elements(by=By.XPATH, value="//div[@class='mt-ListAds']//@href")
        time.sleep(random.uniform(0.2, 0.8))
        links = [element.get_attribute('href') for element in elements]
        if position == 0:
            try:
                actions.move_to_element(elements[0]).perform()
            except:
                pass
            position += 1
        else:
            try:
                actions.move_by_offset(random.uniform(-20, 20), random.uniform(-100, 200))
            except:
                pass
        increment += 300
        if len(elements) >= 30 or increment >= 10000:
            break
    links = [element.get_attribute('href') for element in elements]
    display(len(links))
    display(links)
    return links

def get_car(driver, url):
    car = {}
    actions = ActionChains(driver)
    driver.get(url)
    time.sleep(random.uniform(0.5, 1.5))
    try:
        button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div[2]/div/div[3]/div/div/div/footer/div/button[2]')
        actions.move_to_element(button).perform()
        button.click()
        time.sleep(random.uniform(0.8, 1.2))
    except:
        pass
    car['url'] = url
    car['datetime'] = datetime.now()
    
    car['resto_de_datos'] = get_all_car_info()
    
    try:
        actions.move_by_offset(random.uniform(-20, 20), random.uniform(-100, 100))
    except:
        pass
    time.sleep(random.uniform(1.8, 3.2))
    return car

def get_all_car_info():
  # Busca todas las etiquetas 'script' dentro del 'body'
  script_tags = driver.find_elements(By.XPATH, "//body//script")
  script_content = ""

  for tag in script_tags:
      script = tag.get_attribute("innerHTML")
      if '__INITIAL_PROPS__' in script:
          script_content = script
          break
  # extraer la cadena JSON del texto plano
  match = re.search(r'JSON.parse\("(.*?)"\);', script_content)

  json_string = match.group(1)
  json_string = re.sub(r'\\([\\"])', r'\1', json_string) # eliminar las barras invertidas escapadas
  json_string = json_string.replace('\r', '') # eliminar los caracteres "\r"

  # analizar la cadena JSON
  json_data = json.loads(json_string)

  # Procesar json
  keys_to_extract = [
      "color", "fuelTypeId", "fuelType", "id", "km", "make", "makeId", "model", "modelId", "price", "province",
      "provinceId", "title", "transmissionType", "transmissionTypeId", "year"
  ]

  processed_json = {key: json_data["ad"][key] for key in keys_to_extract}
  processed_json.update({key: json_data["leadData"][key] for key in ["brand", "brand_id", "fuel_types", "model_id", "seller_type"]})
  processed_json.update({key: json_data["vehicleInfo"]["specifications"][key] for key in ["bodyType", "cubicCapacity", "doors", "hp"]})

  processed_json["url"] = 'https://www.coches.net' + json_data["ad"]["url"]

  #print(processed_json)
  return processed_json


In [7]:
driver = get_driver()
actions = ActionChains(driver)
driver.get('https://www.coches.net/segunda-mano/?MakeId=1330')
try:
  actions.move_by_offset(random.uniform(-20, 20), random.uniform(-100, 200))
except:
  pass
time.sleep(random.uniform(0.5, 1.5))
try:
  button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div[2]/div/div[3]/div/div/div/footer/div/button[2]')
  try:
    actions.move_to_element(button).perform()
  except:
    pass
  button.click()
  time.sleep(random.uniform(0.8, 1.2))
except:
  pass

In [ ]:
coche = get_car(driver,'https://www.coches.net/abarth-500c-595c-turismo-14-16v-tjet-121kw-e6d-gasolina-2020-en-salamanca-53762829-covo.aspx')

In [11]:
links = get_links(driver,'https://www.coches.net/segunda-mano/?MakeId=1330')

InvalidSelectorException: Message: invalid selector: The result of the xpath expression "//div[@class='mt-ListAds']//@href" is: [object Attr]. It should be an element.
  (Session info: chrome=112.0.5615.49)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0022DCE3+50899]
	(No symbol) [0x001BE111]
	(No symbol) [0x000C5588]
	(No symbol) [0x000C7EDB]
	(No symbol) [0x000C7DB0]
	(No symbol) [0x000C8030]
	(No symbol) [0x000F0552]
	(No symbol) [0x000F0AFB]
	(No symbol) [0x0011F93C]
	(No symbol) [0x0010B944]
	(No symbol) [0x0011E01C]
	(No symbol) [0x0010B6F6]
	(No symbol) [0x000E7708]
	(No symbol) [0x000E886D]
	GetHandleVerifier [0x00493EAE+2566302]
	GetHandleVerifier [0x004C92B1+2784417]
	GetHandleVerifier [0x004C327C+2759788]
	GetHandleVerifier [0x002C5740+672048]
	(No symbol) [0x001C8872]
	(No symbol) [0x001C41C8]
	(No symbol) [0x001C42AB]
	(No symbol) [0x001B71B7]
	BaseThreadInitThunk [0x767A00F9+25]
	RtlGetAppContainerNamedObjectPath [0x776A7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x776A

In [9]:
links

[]